In [1]:
import pickle
import numpy as np

In [2]:
with open("datadict", "rb") as f:
    d = pickle.load(f)
d.keys()

dict_keys(['time_list', 'a_b_c_I_dt_list', 'g_mat', 'g_cov', 'Xtrue', 'Ytrain', 'Yvalid'])

In [3]:
x = d['Xtrue']
y_train = d['Ytrain']
y_test = d['Yvalid']
x_train = x[:len(y_train)]
x_test = x[len(y_train):]
print(x.shape, y_train.shape, y_test.shape)

(240, 200, 2) (200, 200, 1) (40, 200, 1)


In [4]:
y_train_day = np.tile(np.arange(200)[np.newaxis, :], (len(y_train), 1))
y_train_day = y_train_day[:, :, np.newaxis]
y_train_new = np.concatenate((y_train_day, y_train), axis=-1)
y_test_day = np.tile(np.arange(200)[np.newaxis, :], (len(y_test), 1))
y_test_day = y_test_day[:, :, np.newaxis]
y_test_new = np.concatenate((y_test_day, y_test), axis=-1)
print(y_train_new.shape, y_test_new.shape)
print(y_train_new[0, :10, :])

(200, 200, 2) (40, 200, 2)
[[ 0.         -0.75991914]
 [ 1.         -0.94188864]
 [ 2.         -0.88082116]
 [ 3.         -0.9476135 ]
 [ 4.         -0.98917134]
 [ 5.         -1.01295341]
 [ 6.         -0.69137809]
 [ 7.         -0.74303678]
 [ 8.         -0.52540432]
 [ 9.         -0.37549246]]


In [5]:
# 20 % missing obs
T = y_train_new.shape[1]
for obs_percentage in [0.8, 0.6, 0.5, 0.4]:
    y_train_tmp = []
    y_test_tmp = []
    for y in y_train_new:
        obsed_days = np.random.choice(np.arange(T), int(T * obs_percentage), replace=False)
        obsed_days = np.sort(obsed_days)
        y_train_tmp.append(y[obsed_days])
    for y in y_test_new:
        obsed_days = np.random.choice(np.arange(T), int(T * obs_percentage), replace=False)
        obsed_days = np.sort(obsed_days)
        y_test_tmp.append(y[obsed_days])
    y_train_tmp = np.stack(y_train_tmp)
    y_test_tmp = np.stack(y_test_tmp)
    print(y_train_tmp.shape, y_test_tmp.shape)
    days = np.arange(T)[np.newaxis, :, np.newaxis]
    v_train = np.concatenate([np.tile(days, (len(y_train_new), 1, 1)), np.zeros((len(y_train_new), T, 1))], axis=-1)
    v_test = np.concatenate([np.tile(days, (len(y_test_new), 1, 1)), np.zeros((len(y_test_new), T, 1))], axis=-1)
    with open("fhn_{}_obs.p".format(obs_percentage), "wb") as f:
        pickle.dump({'Ytrain': y_train_tmp, 'Ytest': y_test_tmp, 'Vtrain': v_train, 'Vtest': v_test}, f)

(200, 160, 2) (40, 160, 2)
(200, 120, 2) (40, 120, 2)
(200, 100, 2) (40, 100, 2)
(200, 80, 2) (40, 80, 2)


In [6]:
print(v_train[0, :10])

[[0. 0.]
 [1. 0.]
 [2. 0.]
 [3. 0.]
 [4. 0.]
 [5. 0.]
 [6. 0.]
 [7. 0.]
 [8. 0.]
 [9. 0.]]
